In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import image_interpolation
tf.config.list_physical_devices('GPU')

2022-09-30 18:46:35.914295: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 18:46:36.038894: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 18:46:36.616812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-30 18:46:36.616857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
train,test=tf.keras.datasets.cifar10.load_data()
x_train,y_train=train
x_test,y_test=test
print("input shape: ",x_train.shape)
print("output shape: ",y_train.shape)
x_train[0].max()

input shape:  (50000, 32, 32, 3)
output shape:  (50000, 1)


255

In [3]:
input=tf.keras.Input(shape=(None, None,3))
num_iterations=5
repeated_input = tf.repeat(tf.expand_dims(input,1), repeats=num_iterations, axis=1)
classifier=tf.keras.Sequential([
    tf.keras.layers.RNN(image_interpolation.ImageSectionRNNCell(grid_dim=(8, 18), units=32)),
    tf.keras.layers.Dense(10, activation='softmax')
])
output=classifier(repeated_input)
model=tf.keras.Model(inputs=input, outputs=output)
model.summary()


2022-09-30 18:46:45.097623: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 18:46:45.099235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 18:46:45.099474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 18:46:45.099629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

LSTM state:  [<tf.Tensor 'rnn/zeros:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'rnn/zeros_1:0' shape=(None, 32) dtype=float32>]
section_state:  Tensor("rnn/Repeat/Reshape_1:0", shape=(None, 3), dtype=float32)
LSTM state:  [<tf.Tensor 'sequential_2/rnn/zeros:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'sequential_2/rnn/zeros_1:0' shape=(None, 32) dtype=float32>]
section_state:  Tensor("sequential_2/rnn/Repeat/Reshape_1:0", shape=(None, 3), dtype=float32)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 1, None, None, 3)  0        
                                                                 
 tf.repeat (TFOpLambda)      (None, 5, None, None, 3)  0         
                                                          

In [4]:
with tf.device('/CPU:0'):
    model.compile(optimizer='adam',metrics=['accuracy'], loss='sparse_categorical_crossentropy')
    model.fit(x_train,y_train,epochs=10, batch_size=8)

Epoch 1/10


2022-09-30 18:47:01.279300: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 153600000 exceeds 10% of free system memory.


LSTM state:  [<tf.Tensor 'model/sequential_2/rnn/zeros:0' shape=(8, 32) dtype=float32>, <tf.Tensor 'model/sequential_2/rnn/zeros_1:0' shape=(8, 32) dtype=float32>]
section_state:  Tensor("model/sequential_2/rnn/Repeat/Reshape_1:0", shape=(8, 3), dtype=float32)
LSTM state:  [<tf.Tensor 'model/sequential_2/rnn/zeros:0' shape=(8, 32) dtype=float32>, <tf.Tensor 'model/sequential_2/rnn/zeros_1:0' shape=(8, 32) dtype=float32>]
section_state:  Tensor("model/sequential_2/rnn/Repeat/Reshape_1:0", shape=(8, 3), dtype=float32)
